In [ ]:
!nvidia-smi

Mon Jul  1 11:16:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   33C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!huggingface-cli scan-cache

Cache directory not found: /root/.cache/huggingface/hub


In [ ]:
!pip install -qU \
  transformers==4.41.0\
  sentence-transformers \
  accelerate \
  einops \
  langchain==0.2.1 \
  langchain-openai \
  langchain_cohere \
  cohere \
  tiktoken \
  xformers \
  bitsandbytes \
  chromadb==0.4.15 \
  openai \
  datasets \
  ragas \
  wandb \
  tiktoken \
  rank_bm25

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok pypi-json

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR-API-KEY"
os.environ["COHERE_API_KEY"] = "YOUR-API-KEY"

In [ ]:
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token('YOUR-API-KEY')

In [ ]:
import os
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
import torch
from torch import cuda, bfloat16
import transformers
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from io import BytesIO
from docx import Document
import chromadb
from tqdm import tqdm

# Khởi tạo

In [ ]:
def get_text_chunks(text,size):
    from langchain_core.documents import Document
    if len(text) <= size or len(text) <=100:
        return [Document(page_content=text,metadata={"chunk_size": size,"chunk_index": 0,"total_chunk":1})]

    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n"],
        chunk_size=size,
        chunk_overlap=256,
        add_start_index=True,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    doc_chunks = []
    for i, chunk in enumerate(chunks):
      doc_chunks.append(Document(page_content=chunk,
                                metadata={"chunk_size": size,
                                          "chunk_index": i,
                                          "total_chunk":len(chunks)}))
    return doc_chunks

In [ ]:
def load_embed_model():
    from langchain.embeddings import HuggingFaceBgeEmbeddings
    embed_model_id = 'BAAI/bge-base-en-v1.5'

    #Cấu hình thiết bị
    device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

    embed_model = HuggingFaceBgeEmbeddings(
        model_name=embed_model_id,
        model_kwargs={'device': device},
        encode_kwargs={'device': device, 'batch_size': 32,'normalize_embeddings': True},
        query_instruction="Represent this sentence for searching relevant passages:"

    )
    return embed_model

In [ ]:
def create_vectorstore(embed_model):
  text="Hello World"
  chunks = get_text_chunks(text,size=1024)
  embedding = embed_model
  vectorstore = Chroma.from_documents(chunks, embedding)
  doc_ids = vectorstore.get()["ids"]
  vectorstore.delete(ids=doc_ids)

  return vectorstore

def create_bm25():
  from langchain_core.documents import Document

  retriever = BM25Retriever.from_documents(
      [
          Document(page_content="Hello World",metadata={'idex':1})
      ]
  )

  return retriever

In [ ]:
def find_page_for_chunk(chunk_number,total_pages, total_chunks, chunk_size, chunk_over_lap, page_lenghts):
    effective_chunk_size = chunk_size - chunk_over_lap
    total_effective_chunks = total_chunks - ((total_chunks // chunk_size) * chunk_over_lap)
    chunks_per_page = total_effective_chunks / total_pages
    cumulative_lengths = [sum(page_lenghts[:i+1]) for i in range(total_pages)]
    target_lenght = chunk_number * effective_chunk_size
    page_number = next(i+1 for i,length in enumerate(cumulative_lengths) if length >= target_lenght)
    return page_number

In [ ]:
def is_exist_information(retriever,question):
    result = retriever.invoke(question)
    return result[0].metadata['relevance_score'] > 0.1

## retrieval

In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank


class VectorstoreCenter:
  def __init__(self,embedding,vectorstore,bm25_retriever):
    self.embedding = embedding
    self.vectorstore = vectorstore
    self.bm25_retriever = bm25_retriever

  def viewe_index(self):
    return self.vectorstore.get()["ids"]

  def cleanDB(self):
    doc_ids = self.vectorstore.get()["ids"]
    if len(doc_ids) > 0:
      self.vectorstore.delete(ids=doc_ids)

  def add_text(self,text):
    chunks_small = get_text_chunks(text,size=512)
    chunks_large = get_text_chunks(text,size=512)
    self.vectorstore.from_documents(chunks_small,self.embedding)
    self.bm25_retriever = BM25Retriever.from_documents(chunks_large)

  def add_document(self,chunks_small,chunks_large):
    self.vectorstore.from_documents(chunks_small,self.embedding)
    self.bm25_retriever = BM25Retriever.from_documents(chunks_large)

  def retriver(self,k=4,top_n=4):
    retriever = self.vectorstore.as_retriever(search_kwargs={"k": k})
    self.bm25_retriever.k = k
    ensemble_retriever = EnsembleRetriever(retrievers=[self.bm25_retriever, retriever], weights=[0.4,0.6])
    compressor = CohereRerank()
    compressor.top_n=top_n
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=ensemble_retriever,
    )
    return compression_retriever



# llama3 model

## load model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## pipeline generate

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=512,
    eos_token_id=terminators,
    #pad_token_id=tokenizer.eos_token_id
)

llm_llama = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt_template = """
<|start_header_id|>user<|end_header_id|>
You are an assistant for answering questions.
You are given the extracted parts of a long document and a question. Provide a conversational answer.
I want you to think step by step to answer the query in a crisp manner.
Answer naturally and don't say things like "Based on the context provided" or "According to the text".
And don't repeat the question in your answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


chain_type_kwargs = {"prompt": prompt}




## rag chain

In [ ]:
def create_ragchain(retriever):
  rag_chain = RetrievalQA.from_chain_type(llm=llm_llama,
                                chain_type="stuff",
                                retriever=retriever,
                                chain_type_kwargs=chain_type_kwargs,
                                return_source_documents = True)
  return rag_chain

# API


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
embedding_model = load_embed_model()
vectorstore = create_vectorstore(embedding_model)
bm25retriever = create_bm25()
page_lengths = []
dataCenter = VectorstoreCenter(embedding_model,vectorstore,bm25retriever)
rag_chain = create_ragchain(dataCenter.retriver())

In [ ]:
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel
from fastapi.responses import JSONResponse
import uvicorn
import nest_asyncio

nest_asyncio.apply()

app = FastAPI()
class TextFile(BaseModel):
    text: str
    page_lengths: list


class Answer(BaseModel):
    input: str
    response: dict

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/items/")
async def insert_into_db(text_file: TextFile):
    global retriever
    global page_lengths

    try:
      dataCenter.cleanDB()
      dataCenter.add_text(text_file.text)
      page_lengths = text_file.page_lengths
      retriever = dataCenter.retriver()
      return "Loaded into the database"
    except Exception as e:
      raise HTTPException(status_code=500, detail="Error entering database")

@app.get("/answer/",response_model=Answer)
async def return_answer(question: str):
    global retriever
    global page_lengths
    try:
      if not is_exist_information(retriever,question):
        response =  f"I'm sorry, but I couldn't find information about '{question}' in the documentation provided. If you have any other questions or need information about a specific topic from the document, please ask!"
        return Answer(input=question,response={'answer':response})
      answer = rag_chain.invoke(question)['result']
      sources = retriever.invoke(question)
      origin_pages = []
      try:
        for source in sources[0:3]:
          if source.metadata["relevance_score"] > 0.3:
            chunk_number = source.metadata["chunk_index"]
            total_chunks = source.metadata["total_chunk"]
            total_pages = len(page_lengths)
            chunk_size = source.metadata["chunk_size"]
            chunk_over_lap = 256
            orig = find_page_for_chunk(chunk_number,total_pages,total_chunks,chunk_size,chunk_over_lap,page_lengths)
            origin_pages.append(orig)
        set_pages=sorted(set(origin_pages))
        response = {'answer': answer+"\nSource pages: " + ",".join(map(str,set_pages)) +"."} if len(set_pages) > 0 else {'answer': answer}
      except:
        response = {'answer': answer}
      return Answer(input=question,response=response)
    except Exception as e:
      raise HTTPException(status_code=500, detail="Server Error")

In [ ]:
auth_token = "YOUR-API-KEY"
ngrok.set_auth_token(auth_token)

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print('Public URL docs:', ngrok_tunnel.public_url+'/docs')
nest_asyncio.apply()
uvicorn.run(app, port=8000)